In [1]:
# ADIM 1.1: Kütüphaneleri Yükleme (Tekrar)
!pip install -q langchain langchain-chroma langchain-google-genai datasets pandas streamlit
print("✅ Kütüphaneler yüklendi.")

# ADIM 1.2: API Anahtarını GİRİN
import os
import getpass

# Anahtarı tekrar isteyelim
API_KEY = getpass.getpass("Lütfen Gemini API Anahtarınızı buraya yapıştırın: ")

# Not: Anahtarı os.environ'a da atıyoruz, ancak 3. ve 4. adımlarda direkt kullanacağız.
os.environ["GEMINI_API_KEY"] = API_KEY

print("✅ API Anahtarı yüklendi. (Görünmez kalacaktır)")

✅ Kütüphaneler yüklendi.
Lütfen Gemini API Anahtarınızı buraya yapıştırın: ··········
✅ API Anahtarı yüklendi. (Görünmez kalacaktır)


In [3]:
from datasets import load_dataset
import pandas as pd

# Veri setini yükleme (Sadece ilk 200 satır)
try:
    dataset = load_dataset("Hieu-Pham/kaggle_food_recipes", split="train[:200]")
    df = dataset.to_pandas()
except Exception as e:
    print(f"Hata: Veri seti yüklenirken veya dönüştürülürken sorun oluştu: {e}")
    # Eğer hata alırsanız, bu kısmı kontrol edin.

# Doğru sütun adlarını kullanarak RAG için metin belgesi oluşturma.
# Düzeltilmiş sütun adları: 'Title', 'Ingredients', 'Instructions'
df['full_recipe'] = df.apply(
    lambda row: f"TARİF ADI: {row['Title']}\nMALZEMELER: {', '.join(row['Ingredients'])}\nADIMLAR: {row['Instructions']}",
    axis=1
)

# RAG için kullanılacak belge listesi
recipe_docs = df['full_recipe'].tolist()

print(f"✅ Toplam {len(recipe_docs)} adet tarif belgesi başarıyla hazırlandı.")
print("-" * 50)
print("Örnek Belge Başlangıcı:")
print(recipe_docs[0][:250] + "...")

✅ Toplam 200 adet tarif belgesi başarıyla hazırlandı.
--------------------------------------------------
Örnek Belge Başlangıcı:
TARİF ADI: Miso-Butter Roast Chicken With Acorn Squash Panzanella
MALZEMELER: [, ', 1,  , (, 3, ½, –, 4, -, l, b, ., ),  , w, h, o, l, e,  , c, h, i, c, k, e, n, ', ,,  , ', 2, ¾,  , t, s, p, .,  , k, o, s, h, e, r,  , s, a, l, t, ,,  , d, i, v, i, d...


In [4]:
from langchain_chroma import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.chains import RetrievalQA
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate

# Daha önce getpass ile aldığımız anahtarı kullanıyoruz
if 'API_KEY' not in globals():
    raise ValueError("API_KEY değişkeni bulunamadı. Lütfen 1. Adımı tekrar çalıştırın.")

# 3.1 Embedding Modelini ve Vektör Veritabanını Tanımlama
# DÜZELTME: Anahtarı doğrudan 'google_api_key' parametresi ile geçiyoruz.
embedding_model = GoogleGenerativeAIEmbeddings(
    model="text-embedding-004",
    google_api_key=API_KEY # KESİN ÇÖZÜM BURASI
)

print("Embedding model oluşturuluyor...")
# ChromaDB'yi oluşturma ve belgeleri yükleme (Bu adım Embedding modelini kullanır ve hata bu adımda çıkıyordu)
vectorstore = Chroma.from_texts(
    texts=recipe_docs,
    embedding=embedding_model,
    collection_name="yemek_tarifleri_rag"
)

# Retriever'ı tanımlama
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

# 3.2 LLM ve Prompt Tanımlama
# DÜZELTME: LLM için de anahtarı doğrudan geçiriyoruz.
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.2,
    google_api_key=API_KEY # KESİN ÇÖZÜM BURASI
)

# LLM'ye bağlamı ve talimatı ileten custom prompt
PROMPT_TEMPLATE = """Aşağıdaki bağlamda sana verilen yemek tariflerini kullanarak, kullanıcının sorusuna detaylı ve yardımcı bir şekilde yanıt ver.
Eğer bağlamda uygun tarif bulamazsan, kibarca sadece "Üzgünüm, veri tabanımda bu isteğe uygun bir tarif bulamadım." diye yanıtla ve dışarıdan bilgi ekleme.

BAĞLAM:
{context}

SORU: {question}
YANIT:"""

custom_rag_prompt = PromptTemplate.from_template(PROMPT_TEMPLATE)

# 3.3 RAG Zincirini Kurma
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs={"prompt": custom_rag_prompt},
    return_source_documents=True
)

print("✅ RAG Mimarisi (Embedding, ChromaDB, Retriever, LangChain) başarıyla kuruldu ve API Anahtarı sorunu çözüldü.")

Embedding model oluşturuluyor...
✅ RAG Mimarisi (Embedding, ChromaDB, Retriever, LangChain) başarıyla kuruldu ve API Anahtarı sorunu çözüldü.


In [5]:
%%writefile app.py
import streamlit as st
import os
import pandas as pd
from datasets import load_dataset

# LangChain bileşenleri
from langchain_chroma import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.chains import RetrievalQA
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate

# ----------------------------------------------------------------------
# 1. RAG Bileşenlerini Tanımlama ve Yükleme (Cache ile hızlandırıldı)
# ----------------------------------------------------------------------

# ⚠️ ÖNEMLİ: API Anahtarı, uygulamanın dağıtım ortamında (Streamlit Cloud, vb.)
# Güvenli Depolama (Secrets) yoluyla sağlanmalıdır. Colab'de direkt os.environ'dan alır.
API_KEY = os.environ.get("GEMINI_API_KEY")

if not API_KEY:
    st.error("❌ API Anahtarı bulunamadı. Lütfen Colab'de GEMINI_API_KEY değişkenini ayarlayın.")
    st.stop()


# LLM ve Embedding Modelini Tanımlama (Doğrudan API Anahtarı ile)
@st.cache_resource
def get_llm_model():
    return ChatGoogleGenerativeAI(
        model="gemini-2.5-flash",
        temperature=0.2,
        google_api_key=API_KEY
    )

@st.cache_resource
def get_embedding_model():
    return GoogleGenerativeAIEmbeddings(
        model="text-embedding-004",
        google_api_key=API_KEY
    )

# 2. Veri Seti Yükleme ve Hazırlama (Dağıtım sırasında veriyi uygulamaya dahil eder)
@st.cache_data
def load_and_prepare_data():
    dataset = load_dataset("Hieu-Pham/kaggle_food_recipes", split="train[:200]")
    df = dataset.to_pandas()
    df['full_recipe'] = df.apply(
        lambda row: f"TARİF ADI: {row['Title']}\nMALZEMELER: {', '.join(row['Ingredients'])}\nADIMLAR: {row['Instructions']}",
        axis=1
    )
    return df['full_recipe'].tolist()

# 3. Vektör Veritabanı ve Retriever'ı Yükleme/Oluşturma
@st.cache_resource
def get_retriever(recipe_docs):
    embedding_model = get_embedding_model()
    # ChromaDB'yi oluşturma ve belgeleri yükleme
    vectorstore = Chroma.from_texts(
        texts=recipe_docs,
        embedding=embedding_model,
        collection_name="yemek_tarifleri_rag"
    )
    return vectorstore.as_retriever(search_kwargs={"k": 3})

# 4. RAG Zincirini Kurma
@st.cache_resource
def get_qa_chain(retriever):
    llm = get_llm_model()

    PROMPT_TEMPLATE = """Aşağıdaki bağlamda sana verilen yemek tariflerini kullanarak, kullanıcının sorusuna detaylı ve yardımcı bir şekilde yanıt ver.
    Eğer bağlamda uygun tarif bulamazsan, kibarca sadece "Üzgünüm, veri tabanımda bu isteğe uygun bir tarif bulamadım." diye yanıtla ve dışarıdan bilgi ekleme.

    BAĞLAM:
    {context}

    SORU: {question}
    YANIT:"""
    custom_rag_prompt = PromptTemplate.from_template(PROMPT_TEMPLATE)

    return RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        chain_type_kwargs={"prompt": custom_rag_prompt},
        return_source_documents=True
    )

# ----------------------------------------------------------------------
# 5. Streamlit Uygulama Arayüzü
# ----------------------------------------------------------------------

# Veriyi yükle ve RAG bileşenlerini kur
recipe_docs = load_and_prepare_data()
retriever = get_retriever(recipe_docs)
qa_chain = get_qa_chain(retriever)


st.set_page_config(page_title="Akbank GenAI Yemek Tarifleri Chatbotu", layout="wide")
st.title("🍽️ Akbank GenAI Yemek Tarifleri Chatbotu (RAG)")
st.caption(f"Veri tabanımızda {len(recipe_docs)} tarif bulunmaktadır. (Gemini 2.5 Flash ile güçlendirilmiştir)")
st.divider()

if 'history' not in st.session_state:
    st.session_state.history = []

# Kullanıcı girişi
user_query = st.chat_input("Tarif sorunuzu girin (Örn: Ispanak ve peynirle ne yapabilirim?)")

if user_query:
    # Kullanıcı sorgusunu kaydet
    st.session_state.history.append({"role": "user", "content": user_query})

    with st.spinner(f"'{user_query}' için tarif aranıyor..."):
        try:
            # RAG Zincirini Çalıştırma
            response = qa_chain.invoke({"query": user_query})
            llm_response = response['result']
            source_docs = response['source_documents']

            # Yanıtı ve kaynakları geçmişe ekleme
            st.session_state.history.append({"role": "assistant", "content": llm_response, "sources": source_docs})

        except Exception as e:
            error_msg = f"API hatası: Lütfen API anahtarınızın doğru olduğundan ve servislerin çalıştığından emin olun. Hata: {e}"
            st.session_state.history.append({"role": "assistant", "content": error_msg, "sources": []})

# Geçmişi gösterme
for message in st.session_state.history:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

        if message["role"] == "assistant" and "sources" in message and message["sources"]:
            st.markdown("---")
            st.markdown("**Kullanılan Kaynak Tarifler:**")
            source_names = [doc.page_content.split('\n')[0].replace('TARİF ADI: ', '') for doc in message["sources"]]
            for name in set(source_names):
                st.markdown(f"**-** *{name}*")

Writing app.py


In [15]:
%%writefile app.py
import streamlit as st
import os
import pandas as pd
from datasets import load_dataset
# LangChain bileşenleri
from langchain_chroma import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.chains import RetrievalQA
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate

# ----------------------------------------------------------------------
# 1. API Anahtarının Güvenli Kontrolü
# ----------------------------------------------------------------------

API_KEY = os.environ.get("GEMINI_API_KEY")

if not API_KEY:
    st.error("❌ API Anahtarı bulunamadı. Lütfen Streamlit Cloud'da 'GEMINI_API_KEY' Secret'ını ayarlayın.")
    st.stop()

# ----------------------------------------------------------------------
# 2. RAG Bileşenleri Tanımları (FONKSİYONLAR BURADA BAŞLAR)
# ----------------------------------------------------------------------

# LLM ve Embedding Modelini Tanımlama (Doğrudan API Anahtarı ile)
@st.cache_resource
def get_llm_model():
    return ChatGoogleGenerativeAI(
        model="gemini-2.5-flash",
        temperature=0.2,
        google_api_key=API_KEY
    )

@st.cache_resource
def get_embedding_model():
    return GoogleGenerativeAIEmbeddings(
        model="text-embedding-004",
        google_api_key=API_KEY
    )

# Veri Seti Yükleme ve Hazırlama
@st.cache_data
def load_and_prepare_data():
    dataset = load_dataset("Hieu-Pham/kaggle_food_recipes", split="train[:200]")
    df = dataset.to_pandas()
    df['full_recipe'] = df.apply(
        lambda row: f"TARİF ADI: {row['Title']}\nMALZEMELER: {', '.join(row['Ingredients'])}\nADIMLAR: {row['Instructions']}",
        axis=1
    )
    return df['full_recipe'].tolist()

# Vektör Veritabanı ve Retriever'ı Yükleme/Oluşturma
@st.cache_resource
def get_retriever(recipe_docs):
    embedding_model = get_embedding_model()
    vectorstore = Chroma.from_texts(
        texts=recipe_docs,
        embedding=embedding_model,
        collection_name="yemek_tarifleri_rag"
    )
    return vectorstore.as_retriever(search_kwargs={"k": 3})

# RAG Zincirini Kurma (Cache dekoratörü kalıcı olarak kaldırıldı)
def get_qa_chain(retriever):
    llm = get_llm_model()

    PROMPT_TEMPLATE = """Aşağıdaki bağlamda sana verilen yemek tariflerini kullanarak, kullanıcının sorusuna detaylı ve yardımcı bir şekilde yanıt ver.
    Eğer bağlamda uygun tarif bulamazsan, kibarca sadece "Üzgünüm, veri tabanımda bu isteğe uygun bir tarif bulamadım." diye yanıtla ve dışarıdan bilgi ekleme.

    BAĞLAM:
    {context}

    SORU: {question}
    YANIT:"""
    custom_rag_prompt = PromptTemplate.from_template(PROMPT_TEMPLATE)

    return RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        chain_type_kwargs={"prompt": custom_rag_prompt},
        return_source_documents=True
    )

# ----------------------------------------------------------------------
# 3. Streamlit Uygulama Arayüzü (Ana İşlem) - HER ŞEY BURADA BAŞLAR
# ----------------------------------------------------------------------

# 3.1 RAG Bileşenlerini Yükleme/Kurma
# Bu çağırmalar artık fonksiyon tanımlarının altında yapıldığı için NameError çözüldü.
recipe_docs = load_and_prepare_data()
retriever = get_retriever(recipe_docs)
qa_chain = get_qa_chain(retriever)


# 3.2 Arayüz Başlıkları
st.set_page_config(page_title="Akbank GenAI Yemek Tarifleri Chatbotu", layout="wide")
st.title("🍽️ Akbank GenAI Yemek Tarifleri Chatbotu (RAG)")
st.caption(f"Veri tabanımızda {len(recipe_docs)} tarif bulunmaktadır. (Gemini 2.5 Flash ile güçlendirilmiştir)")
st.divider()

if 'history' not in st.session_state:
    st.session_state.history = []

# Kullanıcı Girişi
user_query = st.chat_input("Tarif sorunuzu girin (Örn: Ispanak ve peynirle ne yapabilirim?)")

if user_query:
    # Kullanıcı sorgusunu kaydet
    st.session_state.history.append({"role": "user", "content": user_query})

    with st.spinner(f"'{user_query}' için tarif aranıyor..."):
        try:
            # RAG Zincirini Çalıştırma
            response = qa_chain.invoke({"query": user_query})
            llm_response = response['result']
            source_docs = response['source_documents']

            # Yanıtı ve kaynakları geçmişe ekleme
            st.session_state.history.append({"role": "assistant", "content": llm_response, "sources": source_docs})

        except Exception as e:
            error_msg = f"API Hatası: Lütfen API anahtarınızın Streamlit Secrets'ta doğru ayarlandığından emin olun. Hata: {e}"
            st.session_state.history.append({"role": "assistant", "content": error_msg, "sources": []})

# Geçmişi gösterme
for message in st.session_state.history:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

        # Yanıtta kullanılan kaynakları göster
        if message["role"] == "assistant" and "sources" in message and message["sources"]:
            st.markdown("---")
            st.markdown("**Kullanılan Kaynak Tarifler:**")
            source_names = [doc.page_content.split('\n')[0].replace('TARİF ADI: ', '') for doc in message["sources"]]
            for name in set(source_names):
                st.markdown(f"**-** *{name}*")

Overwriting app.py


In [18]:
!cat app.py

import streamlit as st
import os
import pandas as pd
from datasets import load_dataset
# LangChain bileşenleri
from langchain_chroma import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.chains import RetrievalQA
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate

# ----------------------------------------------------------------------
# 1. API Anahtarının Güvenli Kontrolü
# ----------------------------------------------------------------------

API_KEY = os.environ.get("GEMINI_API_KEY")

if not API_KEY:
    st.error("❌ API Anahtarı bulunamadı. Lütfen Streamlit Cloud'da 'GEMINI_API_KEY' Secret'ını ayarlayın.")
    st.stop()

# ----------------------------------------------------------------------
# 2. RAG Bileşenleri Tanımları (FONKSİYONLAR BURADA BAŞLAR)
# ----------------------------------------------------------------------

# LLM ve Embedding Modelini Tanımlama (Doğrudan API Anahtarı ile)
@st

In [16]:
import getpass
import subprocess
from pyngrok import ngrok

# Önceki tünelleri temizleyin
!pkill -f streamlit
!pkill -f ngrok

# 1. ngrok authtoken'ı alın
NGROK_TOKEN = getpass.getpass("Lütfen ngrok Authtoken'ınızı buraya yapıştırın: ")

# 2. ngrok'a kimlik doğrulaması yapın
# Bu, ngrok'un artık sizin hesabınızla çalışmasını sağlar.
ngrok.set_auth_token(NGROK_TOKEN)

# 3. Streamlit uygulamasını tekrar başlatın (Önceki kodunuz)
print("Streamlit uygulamasını arka planda başlatıyor...")
subprocess.Popen(
    ["streamlit", "run", "app.py", "--server.port", "8501", "--server.enableCORS", "false"],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE
)

time.sleep(5)

# 4. Tüneli açın (Bu sefer kimlik doğrulama sorunu yaşanmamalı)
try:
    public_url = ngrok.connect(8501)

    print("\n---------------------------------------------------------")
    print("🎉 Streamlit Uygulamanız ÇALIŞIYOR! Geçici Web Linkiniz:")
    print(f"🔗 {public_url}")
    print("---------------------------------------------------------")

except Exception as e:
    print("\n❌ ngrok tüneli kurulamadı.")
    print("Lütfen ngrok token'ınızın doğru olduğundan emin olun.")
    print("Hata detayı:", e)

Lütfen ngrok Authtoken'ınızı buraya yapıştırın: ··········
Streamlit uygulamasını arka planda başlatıyor...

---------------------------------------------------------
🎉 Streamlit Uygulamanız ÇALIŞIYOR! Geçici Web Linkiniz:
🔗 NgrokTunnel: "https://biconically-limnologic-willow.ngrok-free.dev" -> "http://localhost:8501"
---------------------------------------------------------


In [19]:
%%writefile requirements.txt
langchain
langchain-chroma
langchain-google-genai
datasets
pandas
streamlit

Writing requirements.txt


In [20]:
!cat requirements.txt

langchain
langchain-chroma
langchain-google-genai
datasets
pandas
streamlit
